In [12]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [4]:
ratings_df = pd.read_csv('movielens_matrix.csv')
ratings_df.head()

,user id,1,2,3,4,5,6,7,8,9,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
ratings_df.fillna(0, inplace=True)
#ratings_df = ratings_df.astype(int)
ratings_matrix = ratings_df.drop('user id',axis=1).values
#normalize data
ratings_matrix = ratings_matrix/5.0
ratings_matrix.shape

(943, 1682)

In [6]:
#custom loss to not penalize on 0 values in reconstruction
def masked_mse(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)  # 1 where y_true != 0
    squared_error = tf.square(y_true - y_pred)
    masked_se = mask * squared_error
    return tf.reduce_sum(masked_se) / tf.reduce_sum(mask)  # average over non-zero entries

In [7]:
original_dim = ratings_matrix.shape[1]
encoding_dim = 64

# Input: user's interaction vector (sparse or dense)
input_layer = layers.Input(shape=(original_dim,))

# Encoder
encoded = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.1))(input_layer)
#encoded = layers.Dropout(0.5)(encoded)
#encoded = layers.Dense(264, activation='relu')(encoded)
#encoded = layers.Dropout(0.5)(encoded)
encoded = layers.Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(0.00001))(encoded)

# Decoder
#decoded = layers.Dense(264, activation='relu')(encoded)
decoded = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.1))(encoded)
#decoded = layers.Dropout(0.5)(decoded)
output_layer = layers.Dense(original_dim, activation='linear')(decoded)  # use sigmoid if input is binary; linear if ratings

# Model
autoencoder = Model(inputs=input_layer, outputs=output_layer)

#compile the model
autoencoder.compile(optimizer='adam', loss= masked_mse)

2025-05-11 19:58:31.505922: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
#train the model
#callbacks=[early_stopping]
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

autoencoder.fit(ratings_matrix, ratings_matrix,
                epochs=100,
                batch_size=128,
                shuffle=True,
                validation_split=0.2,
                callbacks=[early_stopping])

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 83.7847 - val_loss: 64.3275
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 59.2644 - val_loss: 44.4199
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 40.7328 - val_loss: 29.9412
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 27.3049 - val_loss: 19.7223
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 17.9229 - val_loss: 12.7883
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 11.5887 - val_loss: 8.2100
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 7.4426 - val_loss: 5.2918
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.8109 - val_loss: 3.4656
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 3.1694 - val_loss: 2.3367
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 2.1527 - val_loss: 1.6348
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.5181 - val_loss: 1.1862
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.10

In [9]:
reconstruction_matrix = autoencoder.predict(ratings_matrix)
#scale back
reconstruction_matrix = reconstruction_matrix * 5

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [10]:
print(reconstruction_matrix.shape)
print(reconstruction_matrix[0:5,:])

(943, 1682)
[[ 3.8826857   3.163757    3.1663733  ...  0.10999134  0.29527918
  -0.14589931]
 [ 3.8826857   3.163757    3.1663733  ...  0.10999134  0.29527918
  -0.14589931]
 [ 3.8826857   3.163757    3.1663733  ...  0.10999134  0.29527918
  -0.14589931]
 [ 3.8826857   3.163757    3.1663733  ...  0.10999134  0.29527918
  -0.14589931]
 [ 3.8826857   3.163757    3.1663733  ...  0.10999134  0.29527918
  -0.14589931]]


### Make recommendations
Certain items are recommended alot

In [16]:
#number of items to recommend
top_k = 10
#zero out previously rated items, so they are not recommended
rated_indices = np.argwhere(ratings_matrix)
rows, columns = rated_indices.T
reconstruction_matrix[rows,columns] = 0
#get random user
user = reconstruction_matrix[np.random.randint(reconstruction_matrix.shape[0],size=1),:]
#make recommendations
recs = np.argsort(-user)[0,0:top_k]
print(recs)
print(f'\npredicted ratings: {user[0,recs]}')

[1190 1188 1292 1652 1598 1535 1121  813 1499 1200]

predicted ratings: [4.984741  4.984087  4.9834895 4.982025  4.975872  4.9752083 4.9675407
 4.960335  4.958105  4.9564657]


### Find items with highest predicted ratings

In [17]:
top_k=20
average_scores = reconstruction_matrix.mean(axis=0)
recs = np.argsort(-average_scores)[0:top_k]
print(f'Recommended Items: {recs}')
print(f'Predicted Ratings: {average_scores[recs]}')

Recommended Items: [1652 1598 1535 1190 1188 1292 1121  813 1200 1499 1466 1397 1593 1641
 1462 1448 1250  118 1124 1063]
Predicted Ratings: [4.976701  4.970595  4.969875  4.968951  4.968233  4.967588  4.9622927
 4.955136  4.951244  4.9475427 4.942621  4.4933805 4.479961  4.476504
 4.4676757 4.441796  4.3611827 4.344105  4.243888  4.2178755]
